## NYC Taxi Stream Processing
For the streaming porting, must have data streaming to the topic "demo-message-1".

*Note: If not working, try changing the GROUP_ID and Consumer Group values to reset

### Shared imports and variables
Run this first since most cells below need at least one of these imports or variables

In [0]:
from pyspark.sql.functions import col, desc, regexp_replace, substring, to_date, from_json, explode, expr
from pyspark.sql.types import StructType, StringType

taxi_zone_path = "/mnt/adlsdemo/nyctaxi/lookups/taxi_zone"
taxi_rate_path = "/mnt/adlsdemo/nyctaxi/lookups/taxi_rate_code"
yellow_delta_path = "/mnt/adlsdemo/nyctaxi/tripdata/yellow_delta"

date_format = "yyyy-MM-dd HH:mm:ss"

# Define a schema that Spark understands. This is one of several ways to do it.
trip_schema = (
  StructType()
    .add('VendorID', 'integer')
    .add('tpep_pickup_datetime', 'string')
    .add('tpep_dropoff_datetime', 'string')
    .add('passenger_count', 'integer')
    .add('trip_distance', 'double')
    .add('RatecodeID', 'integer')
    .add('store_and_fwd_flag', 'string')
    .add('PULocationID', 'integer')
    .add('DOLocationID', 'integer')
    .add('payment_type', 'integer')
    .add('fare_amount', 'double')
    .add('extra', 'double')
    .add('mta_tax', 'double')
    .add('tip_amount', 'double')
    .add('tolls_amount', 'double')
    .add('improvement_surcharge', 'double')
    .add('total_amount', 'double')
)


### Simple load of lookup data
Read data from shared databricks folder and save in delta format within Azure Data Lake Storage (ADLS).

In [0]:
# input_df = (
#   spark.read
#     .option("header","true")
#     .option("inferSchema", "true")
#     .csv("/databricks-datasets/nyctaxi/taxizone/taxi_zone_lookup.csv") 
#   )

# df = input_df.withColumnRenamed("service_zone", "ServiceZone")

# df.write.format("delta").mode("overwrite").save(taxi_zone_path)

# display(df)

### Stream Load of incoming data - Trips for 2019-12
Read streaming data from Event Hubs (using Apache Kafka API) and save in the same delta location within Azure Data Lake Storage (ADLS).

In [0]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

run_version = "v1.5"

topic = 'demo-message-1'

# To setup Key Vault backed secret scope for this first time, replace items in url and follow instructions: 
#   https://<databricks-instance>/#secrets/createScopeSetup

# Password is really a Event Hub connection string, for example -> Endpoint=sb://<namespace>.servicebus.windows.net/;SharedAccessKeyName=ReadWriteTmp;SharedAccessKey=vhNXxXXXXXxxxXXXXXXXxx=;EntityPath=demo-message-1
password = dbutils.secrets.get(scope = "demo", key = "eh-sasl-{0}".format(topic))

EH_SASL = 'kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="{0}";'.format(password)
GROUP_ID = f'tst-group-{run_version}'

consumer_config = {
    'kafka.bootstrap.servers': 'dustin-demo-eh.servicebus.windows.net:9093',
    'kafka.security.protocol': 'SASL_SSL',
    'kafka.sasl.mechanism': 'PLAIN',
    'kafka.group.id': GROUP_ID,
    'kafka.request.timeout.ms': "60000",
    'kafka.session.timeout.ms': "20000",
    'kafka.heartbeat.interval.ms': "10000",
    'kafka.sasl.jaas.config': EH_SASL,
    'subscribe': topic
}

# Read from Kafka, format will be a kafka record
input_df = spark.readStream.format("kafka").options(**consumer_config).load()

# Cast just the value as a string (instead of bytes) then use from_json to convert to an object matching the schema
json_df = (
  input_df.select(
    from_json(col('value').cast('string'), trip_schema).alias("json")
  )
)

# Select all attribues from json as individual columns, cast trip_distance, add columns
transformed_df = (
    json_df
      .select("json.*")
      .withColumn("year_month", regexp_replace(substring("tpep_pickup_datetime",1,7), '-', '_'))
      .withColumn("pickup_dt", to_date("tpep_pickup_datetime", date_format)) 
      .withColumn("dropoff_dt", to_date("tpep_dropoff_datetime", date_format))
      .withColumn("tip_pct", col("tip_amount") / col("total_amount"))
)

# Join in lookup data
zone_df = spark.read.format("delta").load(taxi_zone_path)
trip_df = (
   transformed_df
     .join(zone_df, transformed_df["PULocationId"] == zone_df["LocationID"], how="left").drop("LocationID")
     .withColumnRenamed("Burough", "PickupBurrough")
     .withColumnRenamed("Zone", "PickupZone")
     .withColumnRenamed("ServiceZone", "PickupServiceZone")
)

In [0]:
yellow_delta_path_2 = yellow_delta_path + "_" + run_version

(
 trip_df.writeStream
  .format("delta")
  .outputMode("append")
  .option("checkpointLocation", f"/delta/events/_checkpoints/streaming_demo_{run_version}")
  .partitionBy("year_month")
  .start(yellow_delta_path_2)
)

In [0]:
# Read data out of delta table
delta_stream_df = spark.readStream.format("delta").load(yellow_delta_path_2)
display(delta_stream_df)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,year_month,pickup_dt,dropoff_dt,tip_pct,Borough,PickupZone,PickupServiceZone
1,2019-12-01 00:19:48,2019-12-01 00:24:18,1,0.9,1,N,148,4,1,5.5,3.0,0.5,1.85,0.0,0.3,11.15,2019_12,2019-12-01,2019-12-01,0.16591928251121077,Manhattan,Lower East Side,Yellow Zone
1,2019-12-01 00:42:19,2019-12-01 00:50:34,0,1.4,1,N,79,107,2,7.5,3.0,0.5,0.0,0.0,0.3,11.3,2019_12,2019-12-01,2019-12-01,0.0,Manhattan,East Village,Yellow Zone
1,2019-12-01 00:05:27,2019-12-01 00:16:32,2,1.6,1,N,161,237,2,9.0,3.0,0.5,0.0,0.0,0.3,12.8,2019_12,2019-12-01,2019-12-01,0.0,Manhattan,Midtown Center,Yellow Zone
1,2019-12-01 00:58:51,2019-12-01 01:08:37,2,1.0,1,N,161,230,2,6.5,3.0,0.5,0.0,0.0,0.3,10.3,2019_12,2019-12-01,2019-12-01,0.0,Manhattan,Midtown Center,Yellow Zone
1,2019-12-01 00:14:19,2019-12-01 00:27:06,0,1.7,1,Y,164,163,2,10.0,3.0,0.5,0.0,0.0,0.3,13.8,2019_12,2019-12-01,2019-12-01,0.0,Manhattan,Midtown South,Yellow Zone
1,2019-12-01 00:29:35,2019-12-01 00:32:29,0,0.5,1,N,79,224,1,4.0,3.0,0.5,1.55,0.0,0.3,9.35,2019_12,2019-12-01,2019-12-01,0.16577540106951874,Manhattan,East Village,Yellow Zone
1,2019-12-01 00:36:16,2019-12-01 00:53:42,3,5.5,1,N,79,226,1,18.0,3.0,0.5,4.35,0.0,0.3,26.15,2019_12,2019-12-01,2019-12-01,0.16634799235181644,Manhattan,East Village,Yellow Zone
2,2019-11-30 23:54:45,2019-12-01 00:09:57,1,2.12,1,N,234,230,2,12.0,0.5,0.5,0.0,0.0,0.3,15.8,2019_11,2019-11-30,2019-12-01,0.0,Manhattan,Union Sq,Yellow Zone
2,2019-12-01 00:39:43,2019-12-01 00:52:06,1,1.15,1,N,186,48,2,9.0,0.5,0.5,0.0,0.0,0.3,12.8,2019_12,2019-12-01,2019-12-01,0.0,Manhattan,Penn Station/Madison Sq West,Yellow Zone
2,2019-12-01 00:43:02,2019-12-01 01:11:18,1,13.07,1,N,41,51,2,38.5,0.5,0.5,0.0,0.0,0.3,39.8,2019_12,2019-12-01,2019-12-01,0.0,Manhattan,Central Harlem,Boro Zone


In [0]:
# Example of grouping the data in a stream, which requires defining a window
from pyspark.sql.functions import window, sum, avg, substring

delta_stream_2_df = spark.readStream.format("delta").load(yellow_delta_path_2)
grouped_df = (
      delta_stream_2_df
        .groupBy(
           col("passenger_count"),
           window("tpep_pickup_datetime", "20 seconds"))
        .agg(sum("trip_distance").alias("trip_distance_sum"), avg("tip_pct").alias("tip_pct_avg"))
        .sort(desc("window.start"))
      )

last20_df = grouped_df.withColumn("windowStart", substring(col("window.start"), 1,19)).limit(20)
display(last20_df)

passenger_count,window,trip_distance_sum,tip_pct_avg,windowStart
1,"List(2019-12-01T01:01:40.000+0000, 2019-12-01T01:02:00.000+0000)",5.5,0.16666666666666666,2019-12-01 01:01:40
2,"List(2019-12-01T01:00:20.000+0000, 2019-12-01T01:00:40.000+0000)",3.07,0.08928571428571429,2019-12-01 01:00:20
1,"List(2019-12-01T01:00:00.000+0000, 2019-12-01T01:00:20.000+0000)",4.75,0.08333333333333333,2019-12-01 01:00:00
0,"List(2019-12-01T00:59:40.000+0000, 2019-12-01T01:00:00.000+0000)",0.6,0.1941747572815534,2019-12-01 00:59:40
4,"List(2019-12-01T00:59:40.000+0000, 2019-12-01T01:00:00.000+0000)",0.5,0.0,2019-12-01 00:59:40
1,"List(2019-12-01T00:59:40.000+0000, 2019-12-01T01:00:00.000+0000)",15.75,0.17473869379357185,2019-12-01 00:59:40
3,"List(2019-12-01T00:59:20.000+0000, 2019-12-01T00:59:40.000+0000)",0.84,0.12048192771084336,2019-12-01 00:59:20
2,"List(2019-12-01T00:59:20.000+0000, 2019-12-01T00:59:40.000+0000)",2.9699999999999998,0.08293838862559241,2019-12-01 00:59:20
2,"List(2019-12-01T00:59:00.000+0000, 2019-12-01T00:59:20.000+0000)",20.18,0.16666666666666666,2019-12-01 00:59:00
1,"List(2019-12-01T00:59:00.000+0000, 2019-12-01T00:59:20.000+0000)",1.2,0.0,2019-12-01 00:59:00


## Alternative method to group and view in notebook
You could use a memory output format instead and run sql queries against the streaming query, but that will not continuosly update the chart in the notebook (so you will need to hit refresh on the queries to see the changes).

In [0]:
query = (
  grouped_df
    .writeStream
    .format("memory")        # memory = store in-memory table 
    .queryName("counts")     # counts = name of the in-memory table
    .outputMode("complete")  # complete = all the counts should be in the table
    .start()
)


In [0]:
%sql select date_format(window.end, "MMM-dd HH:mm:ss") as time, tip_pct_avg from counts order by time limit 20

In [0]:
# Read data out of delta table
delta_stream_df = spark.readStream.format("delta").load(yellow_delta_path_2)
display(delta_stream_df)

## Azure Storage as a destination
* One option for streaming output is to write directly to you data lake storage (Azure Data Lake Storage Gen 2 or standard Azure Blob Storage).
* Databricks Delta / Delta Lake file format makes this more efficient, but could do with Parquet, Avro or other formats.

### Alternatively: Send transformed data to Event Hubs for next steps in pipeline

In [0]:
topic2 = 'demo-message-transformed'

password = dbutils.secrets.get("data-lake-demo", "eh-sasl-{0}".format(topic2))

EH_SASL = 'kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="{0}";'.format(password)

producer_config = {
    'kafka.bootstrap.servers': 'dustin-demo-eh.servicebus.windows.net:9093',
    'kafka.security.protocol': 'SASL_SSL',
    'kafka.sasl.mechanism': 'PLAIN',
    'kafka.request.timeout.ms': "60000",
    'kafka.session.timeout.ms': "30000",
    'kafka.sasl.jaas.config': EH_SASL,
    'topic': topic2
}

kafka_output_df = trip_df.selectExpr(
    "CAST(VendorId as STRING) as key",
    "to_json(struct(*)) as value")

# display(kafka_output_df)
kafka_output_df.writeStream \
  .format("kafka") \
  .options(**producer_config) \
  .option("checkpointLocation", f"/delta/events/_checkpoints/cp_{run_version}") \
  .start()

## Tests and other alternative syntax
Collection of examples not used in demo but show other capabilities and syntax.

In [0]:
# Example of grouping the data in a stream, which requires defining a window

from pyspark.sql.functions import window  

grouped_df = (
      trip2_df
        .groupBy(
           col("passenger_count"),
           window("tpep_pickup_datetime", "10 minutes"))
        .sum("trip_distance")
      )

display(grouped_df)

In [0]:
# # Example to get actual json schema from sample file (avoid typing it all manually)
# file_location = "/mnt/blobdemo/nyc_taxi_raw/yellow_tripdata_2018-01.csv"
# sample_df = spark.read \
#     .option('sep',',') \
#     .option("inferSchema","true") \
#     .option("header", "true") \
#     .csv(file_location)
# json_schema = sample_df.schema

In [0]:
transformed_df = spark.read.format("delta").load(yellow_delta_path).limit(1000)
zone_df = spark.read.format("delta").load(taxi_zone_path)
trip_df = transformed_df.join(zone_df, transformed_df.PULocationID == zone_df.LocationID, how="left").drop("LocationID")
display(trip_df)

In [0]:
GROUP_ID = f"test-group-2-{run_version}"
consumer2_config = {
    'kafka.bootstrap.servers': 'dustin-demo-eh.servicebus.windows.net:9093',
    'kafka.security.protocol': 'SASL_SSL',
    'kafka.sasl.mechanism': 'PLAIN',
    'kafka.request.timeout.ms': "60000",
    'kafka.session.timeout.ms': "20000",
    'kafka.sasl.jaas.config': EH_SASL,
    'kafka.group.id': GROUP_ID,
    'subscribe': topic2
}

df3 = spark.readStream \
    .format("kafka") \
    .options(**consumer2_config) \
    .load()

df3 = df3.select(col("value").cast("string"))

# display(df3)

In [0]:
# output_path = f"/mnt/adlsdemo/streaming_output_{run_version}"

# delta_stream_df = spark.readStream.format("delta").load(output_path)
# delta_stream_df.writeStream.format("delta").option("checkpointLocation", "/mnt/adlsdemo/checkpoints/demo_v1").start(output_path+"2")
#  display(delta_stream_df)

In [0]:
df = (
  spark.read
    .option("header","true")
    .option("inferSchema", "true")
    .csv("/databricks-datasets/nyctaxi/taxizone/taxi_rate_code.csv") 
  )

df.write.format("delta").mode("overwrite").save(taxi_rate_path)

display(df)

In [0]:
input_df = (
  spark.read
    .option("header","true")
    .schema(trip_schema)
    .csv("dbfs:/databricks-datasets/nyctaxi/tripdata/yellow/yellow_tripdata_2019-12.csv.gz")
)

input_df.coalesce(1).write.json("dbfs:/data/nyctaxi/december_json")

In [0]:
%fs ls /mnt/adlsdemo/streaming_output_v20/

#### Possible error  
Caused by: org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to recognize 'YYYY-MM-dd hh:mm:ss' pattern in the DateTimeFormatter. 1) You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0. 2) You can form a valid datetime pattern with the guide from https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html